In [1]:
# Data manipulation and analysis
import pandas as pd

# File and system operations
import os

# YAML file handling
import yaml

# String to data structure conversion
import ast

# Date calculations
from dateutil.relativedelta import relativedelta

In [ ]:
#########################################################
# Initial Data Loading and YAML to CSV Conversion
#########################################################

# Set up input and output file paths
# Using relative paths for better portability when sharing
executive_yaml = 'congress-legislators/executive.yaml'  # Source data file
executive_csv = 'executive.csv'  # Intermediate CSV file

# Load and parse YAML file
# The file contains nested data about U.S. presidents and vice presidents
with open(executive_yaml, 'r') as f:
    data = yaml.safe_load(f)

# Process YAML data based on its structure
# The data can be either:
# 1. A list of dictionaries (each dict represents one person)
# 2. A dictionary of lists (needs flattening)
if isinstance(data, list) and all(isinstance(i, dict) for i in data):
    # Case 1: Simple list of records - convert directly to DataFrame
    df = pd.DataFrame(data)
elif isinstance(data, dict) and all(isinstance(v, list) for v in data.values()):
    # Case 2: Nested structure - flatten and combine all records
    df = pd.concat(
        [pd.DataFrame(records).assign(group_key=key) 
         for key, records in data.items() 
         if isinstance(records, list)],
        ignore_index=True
    )
else:
    raise ValueError("Unsupported YAML structure - Expected either list of dicts or dict of lists")

# Create output directory if needed and save to CSV
# This creates an intermediate CSV file for further processing
os.makedirs(os.path.dirname(executive_csv), exist_ok=True)
df.to_csv(executive_csv, index=False)

In [ ]:
# Read the intermediate CSV file using relative path for portability
executive = pd.read_csv('executive.csv')

In [4]:
# Helper function to safely parse string to dict/list
def safe_parse(value, default_value):
    """Parse string representation of dict/list, return default if parsing fails"""
    if pd.isna(value):
        return default_value
    if not isinstance(value, str):
        return value
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        return default_value

# Parse column mapping configurations
NAME_MAPPING = {
    'first': 'first_name',
    'last': 'last_name', 
    'middle': 'middle_name',
    'nickname': 'nickname',
    'suffix': 'suffix'
}

BIO_MAPPING = {
    'birthday': ('birthdate', lambda x: pd.to_datetime(x, errors='coerce').date() if x else None),
    'gender': ('gender', lambda x: x)
}

TERM_MAPPING = {
    'type': ('type', lambda x: x),
    'start': ('start_term', lambda x: pd.to_datetime(x, errors='coerce').date() if x else None),
    'end': ('end_term', lambda x: pd.to_datetime(x, errors='coerce').date() if x else None),
    'party': ('party', lambda x: x),
    'how': ('how', lambda x: x)
}

def parse_data(data, mapping):
    """Generic parser for dictionary data using mapping configuration"""
    if isinstance(data, dict):
        return {new_key: transform(data.get(old_key)) 
                for old_key, (new_key, transform) in mapping.items()} \
                if isinstance(mapping, dict) and any(isinstance(v, tuple) for v in mapping.values()) \
                else {new_key: data.get(old_key) 
                      for old_key, new_key in mapping.items()}
    return {new_key: None for new_key in (dict(mapping).values() if isinstance(mapping, dict) 
            else [v[0] for v in mapping.values()])}

def parse_terms(terms, name_data, bio_data):
    """Parse terms data and combine with name and bio information"""
    terms_list = safe_parse(terms, [])
    if not isinstance(terms_list, list):
        return pd.DataFrame()
    
    # Create DataFrame from terms with transformed data
    terms_df = pd.DataFrame([
        {**parse_data(term, TERM_MAPPING),
         **name_data,
         **bio_data}
        for term in terms_list
        if isinstance(term, dict)
    ])
    return terms_df

# Process the executive DataFrame
# 1. Parse name and bio columns
executive = pd.concat([
    executive,
    executive['name'].apply(lambda x: parse_data(safe_parse(x, {}), NAME_MAPPING)).apply(pd.Series),
    executive['bio'].apply(lambda x: parse_data(safe_parse(x, {}), BIO_MAPPING)).apply(pd.Series)
], axis=1)

# 2. Expand terms
expanded_terms = []
for _, row in executive.iterrows():
    name_data = {k: row.get(k) for k in NAME_MAPPING.values()}
    bio_data = {k: row.get(k) for k, _ in BIO_MAPPING.values()}
    expanded_terms.append(parse_terms(row['terms'], name_data, bio_data))

# 3. Create final DataFrame
executive = pd.concat(expanded_terms, ignore_index=True)

# 4. Clean up unnecessary columns
columns_to_drop = ['id', 'name', 'bio', 'terms']
executive = executive.loc[:, ~executive.columns.duplicated()]
executive = executive.drop(columns=[col for col in columns_to_drop if col in executive.columns])

In [5]:
# Function to create full name from components
def create_full_name(row):
    components = [
        row['first_name'],
        f'"{row["nickname"]}"' if row['nickname'] else None,  # Wrap nickname in quotes if present
        row['middle_name'],
        row['last_name'],
        row['suffix']
    ]
    # Filter out None values and join with spaces
    return ' '.join(filter(pd.notnull, components)) or None

# Create the full_name column
executive['full_name'] = executive.apply(create_full_name, axis=1)

In [ ]:
# Function to create birthday from birthdate
def extract_birthday(birthdate):
    """Extract birthday (month-day) from a birthdate.
    
    Args:
        birthdate (datetime.date): The full birthdate
    
    Returns:
        str: Birthday in 'MM-DD' format, or None if birthdate is invalid
    """
    return birthdate.strftime('%m-%d') if pd.notnull(birthdate) else None

# Create a 'birthday' column from the 'birthdate' column
executive['birthday'] = executive['birthdate'].map(extract_birthday)

print(executive)

In [7]:
def get_zodiac_sign(birthday):
    """
    Determine the zodiac sign based on the birthday (month and day).
    :param birthday: A string in 'MM-DD' format representing the birthday.
    :return: A string representing the zodiac sign.
    """
    if not birthday:
        return None  # Handle missing or invalid birthdays

    # Split the birthday into month and day
    try:
        month, day = map(int, birthday.split('-'))
    except ValueError:
        return None  # Return None if the birthday format is invalid

    # Zodiac sign date ranges
    if (month == 1 and day >= 20) or (month == 2 and day <= 18):
        return "Aquarius"
    elif (month == 2 and day >= 19) or (month == 3 and day <= 20):
        return "Pisces"
    elif (month == 3 and day >= 21) or (month == 4 and day <= 19):
        return "Aries"
    elif (month == 4 and day >= 20) or (month == 5 and day <= 20):
        return "Taurus"
    elif (month == 5 and day >= 21) or (month == 6 and day <= 20):
        return "Gemini"
    elif (month == 6 and day >= 21) or (month == 7 and day <= 22):
        return "Cancer"
    elif (month == 7 and day >= 23) or (month == 8 and day <= 22):
        return "Leo"
    elif (month == 8 and day >= 23) or (month == 9 and day <= 22):
        return "Virgo"
    elif (month == 9 and day >= 23) or (month == 10 and day <= 22):
        return "Libra"
    elif (month == 10 and day >= 23) or (month == 11 and day <= 21):
        return "Scorpio"
    elif (month == 11 and day >= 22) or (month == 12 and day <= 21):
        return "Sagittarius"
    elif (month == 12 and day >= 22) or (month == 1 and day <= 19):
        return "Capricorn"

    return None  # Default case (shouldn't happen)

# Apply the zodiac sign function to the 'birthday' column
executive['zodiac_sign'] = executive['birthday'].apply(get_zodiac_sign)

In [8]:
# zodiac sign color mapping
# this dictionary maps each zodiac sign to a specific color (hex code).
# colors are chosen to represent traits commonly associated with each sign.

zodiac_colors = {
    'Aries': '#FF0000',       # red for Aries (bold and energetic)
    'Taurus': '#008000',      # green for Taurus (grounded and earthy)
    'Gemini': '#FFFF00',      # yellow for Gemini (bright and lively)
    'Cancer': '#00008B',      # dark blue for Cancer (deep and emotional)
    'Leo': '#FFA500',         # orange for Leo (warm and vibrant)
    'Virgo': '#A52A2A',       # brown for Virgo (practical and grounded)
    'Libra': '#FFB6C1',       # light pink for Libra (harmonious and gentle)
    'Scorpio': '#000000',     # black for Scorpio (mysterious and intense)
    'Sagittarius': '#800080', # purple for Sagittarius (adventurous and wise)
    'Capricorn': '#556B2F',   # olive green for Capricorn (disciplined and stable)
    'Aquarius': '#0000FF',    # blue for Aquarius (innovative and free-spirited)
    'Pisces': '#40E0D0',      # turquoise for Pisces (dreamy and intuitive)
    'Unknown': '#D3D3D3'      # light gray for unknown or missing zodiac signs
}

# helper function to get the color for a zodiac sign
def get_zodiac_color(sign):
    """
    retrieve the color associated with a zodiac sign.
    
    parameters:
        sign (str): The zodiac sign (e.g., 'Aries', 'Taurus').
    
    returns:
        str: The hex color code for the zodiac sign, or the default color for 'Unknown'.
    """
    return zodiac_colors.get(sign, zodiac_colors['Unknown'])

# Add a new column for zodiac colors in the executive DataFrame
executive['zodiac_color'] = executive['zodiac_sign'].apply(get_zodiac_color)

In [9]:
# zodiac elements
def get_zodiac_element(zodiac_sign):
    elements = {
        'Aries': 'Fire',
        'Taurus': 'Earth',
        'Gemini': 'Air',
        'Cancer': 'Water',
        'Leo': 'Fire',
        'Virgo': 'Earth',
        'Libra': 'Air',
        'Scorpio': 'Water',
        'Sagittarius': 'Fire',
        'Capricorn': 'Earth',
        'Aquarius': 'Air',
        'Pisces': 'Water'
    }
    return elements.get(zodiac_sign, None)

# Apply the zodiac element function to the 'zodiac_element' column
executive['zodiac_element'] = executive['zodiac_sign'].apply(get_zodiac_element)

In [10]:
# zodiac_element color mapping
# this dictionary maps each zodiac element to a specific color (hex code).
# colors are chosen to represent traits commonly associated with each element.
zodiac_element_colors = {
    'Fire': '#FF4500',   # orange-red for Fire (passionate and energetic)
    'Earth': '#8B4513',  # saddle brown for Earth (stable and grounded)
    'Air': '#87CEEB',    # sky blue for Air (light and free-spirited)
    'Water': '#4682B4'   # steel blue for Water (deep and emotional)
}
# helper function to get the color for a zodiac element
def get_zodiac_element_color(element):
    """
    retrieve the color associated with a zodiac element.
    
    parameters:
        element (str): The zodiac element (e.g., 'Fire', 'Earth').
    
    returns:
        str: The hex color code for the zodiac element, or None if not found.
    """
    return zodiac_element_colors.get(element, None)
# Add a new column for zodiac element colors in the executive DataFrame
executive['zodiac_element_color'] = executive['zodiac_element'].apply(get_zodiac_element_color)

In [11]:
def calculate_term_ages(row):
    try:
        # Extract relevant dates
        start_date = row['start_term']
        end_date = row['end_term']
        birth_date = row['birthdate']

        # Validate dates
        if pd.isna(start_date) or pd.isna(end_date) or pd.isna(birth_date):
            return {'age_start': None, 'age_end': None}

        # Calculate age at the start of the term
        delta_start = relativedelta(start_date, birth_date)
        age_start = max(delta_start.years, 0)  # Ensure age is not negative

        # Calculate age at the end of the term
        delta_end = relativedelta(end_date, birth_date)
        age_end = max(delta_end.years, 0)  # Ensure age is not negative

        return {'age_start': age_start, 'age_end': age_end}
    except Exception as e:
        print(f"Error calculating ages for row: {row}, error: {e}")
        return {'age_start': None, 'age_end': None}

# Apply the function to calculate ages
ages = executive.apply(lambda row: pd.Series(calculate_term_ages(row)), axis=1)

# Combine the calculated ages with the original DataFrame
executive = pd.concat([executive, ages], axis=1)

In [12]:
def calculate_term_duration(row):
    try:
        # Extract relevant dates
        start_date = row['start_term']
        end_date = row['end_term']

        # Calculate the total duration in days
        duration_days_total = (end_date - start_date).days

        # Calculate the difference in years, months, and days using relativedelta
        delta = relativedelta(end_date, start_date)

        # Format the duration as strings
        duration_years_months_str = f"{delta.years} year{'s' if delta.years != 1 else ''}, {delta.months} month{'s' if delta.months != 1 else ''}"
        duration_years_months_days_str = f"{delta.years} year{'s' if delta.years != 1 else ''}, {delta.months} month{'s' if delta.months != 1 else ''}, {delta.days} day{'s' if delta.days != 1 else ''}"

        # Calculate total duration in years as a float
        total_duration_years = round(duration_days_total / 365.25, 2)

        return {
            'duration_days': duration_days_total,
            'total_duration_years': total_duration_years,
            'duration_years_months': duration_years_months_str,
            'duration_years_months_days': duration_years_months_days_str
        }
    except Exception as e:
        print(f"Error calculating duration for row: {row}, error: {e}")
        return {
            'duration_days': None,
            'total_duration_years': None,
            'duration_years_months': None,
            'duration_years_months_days': None
        }

# Apply the function to calculate term durations
term_durations = executive.apply(lambda row: pd.Series(calculate_term_duration(row)), axis=1)

# Combine the calculated durations with the original DataFrame
executive = pd.concat([executive, term_durations], axis=1)

In [21]:
# Define columns to keep with detailed name information
columns_to_keep = [
    'full_name', 'gender', 'birthdate', 'birthday', 'zodiac_sign', 'zodiac_color', 'zodiac_element', 'zodiac_element_color',
    'type', 'party', 'start_term', 'end_term', 'age_start', 'age_end', 'duration_years_months_days', 'total_duration_years'
 ]

# Create a new DataFrame with only the specified columns
filtered_executive = executive[columns_to_keep]

In [22]:
# Save the cleaned DataFrame
filtered_executive.to_csv('cleaned_executive.csv', index=False)

In [23]:
filtered_executive

,full_name,gender,birthdate,birthday,zodiac_sign,zodiac_color,zodiac_element,zodiac_element_color,type,party,start_term,end_term,age_start,age_start,age_end,age_end,duration_years_months_days,duration_years_months_days,total_duration_years,total_duration_years
0,George Washington,M,1732-02-22,02-22,Pisces,#40E0D0,Water,#4682B4,prez,no party,1789-04-30,1793-03-04,57,57,61,61,"3 years, 10 months, 4 days","3 years, 10 months, 4 days",3.84,3.84
1,George Washington,M,1732-02-22,02-22,Pisces,#40E0D0,Water,#4682B4,prez,no party,1793-03-04,1797-03-04,61,61,65,65,"4 years, 0 months, 0 days","4 years, 0 months, 0 days",4.00,4.00
2,John Adams,M,1735-10-19,10-19,Libra,#FFB6C1,Air,#87CEEB,viceprez,Federalist,1789-04-21,1793-03-04,53,53,57,57,"3 years, 10 months, 11 days","3 years, 10 months, 11 days",3.87,3.87
3,John Adams,M,1735-10-19,10-19,Libra,#FFB6C1,Air,#87CEEB,viceprez,Federalist,1793-03-04,1797-03-04,57,57,61,61,"4 years, 0 months, 0 days","4 years, 0 months, 0 days",4.00,4.00
4,John Adams,M,1735-10-19,10-19,Libra,#FFB6C1,Air,#87CEEB,prez,Federalist,1797-03-04,1801-03-04,61,61,65,65,"4 years, 0 months, 0 days","4 years, 0 months, 0 days",4.00,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,Mike Pence,M,1959-06-07,06-07,Gemini,#FFFF00,Air,#87CEEB,viceprez,Republican,2017-01-20,2021-01-20,57,57,61,61,"4 years, 0 months, 0 days","4 years, 0 months, 0 days",4.00,4.00
127,Donald J. Trump,M,1946-06-14,06-14,Gemini,#FFFF00,Air,#87CEEB,prez,Republican,2017-01-20,2021-01-20,70,70,74,74,"4 years, 0 months, 0 days","4 years, 0 months, 0 days",4.00,4.00
128,Donald J. Trump,M,1946-06-14,06-14,Gemini,#FFFF00,Air,#87CEEB,prez,Republican,2025-01-20,2029-01-20,78,78,82,82,"4 years, 0 months, 0 days","4 years, 0 months, 0 days",4.00,4.00
129,Kamala D. Harris,F,1964-10-20,10-20,Libra,#FFB6C1,Air,#87CEEB,viceprez,Democrat,2021-01-20,2025-01-20,56,56,60,60,"4 years, 0 months, 0 days","4 years, 0 months, 0 days",4.00,4.00


In [24]:
filtered_executive.isnull().sum()

full_name                     0
gender                        0
birthdate                     0
birthday                      0
zodiac_sign                   0
zodiac_color                  0
zodiac_element                0
zodiac_element_color          0
type                          0
party                         0
start_term                    0
end_term                      0
age_start                     0
age_start                     0
age_end                       0
age_end                       0
duration_years_months_days    0
duration_years_months_days    0
total_duration_years          0
total_duration_years          0
dtype: int64